<font size=7 face="Courier">Maxwell Raw Data Feed - Source Code

This notebook is used to create a raw data feed form the MaxOne device. Code comes from [read_raw.py](/edit/Projects/Maxwell_Controller/closed_loop_2/edited_code/read_raw.py)

# <font color="grey"> Set Up Notebook

Import packages Alex's packages

In [ ]:
import sys
import zmq
import struct
import array
import numpy as np

Import time. While debugging, we only show data every second so as to not flood the printed output.

In [ ]:
import time

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#%matplotlib inline

# <font color="grey"> Helper Function - Socket Subscriber

Generate a subscriber socket with the same settings as the original C++ program.

In [ ]:
def make_socket(port):
    '''
    Generate a subscriber socket with the same settings as the
    original C++ program.
    '''

    print(f'Acquiring socket {port}... ', end='', flush=True)
    subscriber = ctx.socket(zmq.SUB)
    subscriber.setsockopt(zmq.RCVHWM, 0)
    subscriber.setsockopt(zmq.RCVBUF, 10*20000*1030)
    subscriber.setsockopt_string(zmq.SUBSCRIBE, "")
    subscriber.setsockopt(zmq.RCVTIMEO, 100)
    subscriber.connect(f'tcp://localhost:{port}')

    more = True
    while more:
        try:
            msg = subscriber.recv()
        except zmq.ZMQError as e:
            continue
        more = subscriber.getsockopt(zmq.RCVMORE)
    #print('acquired.', flush=True)

    return subscriber

# <font color="blue">Raw Data Reader Function

In [ ]:

ctx = zmq.Context.instance()

filt = make_socket(7205)
raw = make_socket(7204)
time_old = round( time.time() )

while True:
    # Sometimes the publisher will be interrupted, so don't let that
    # crash the entire program, just skip the frame.
    frame_number = frame_raw = events_data = filt_data = raw_data = weird_data = None
    try:
        # The first component of each message is the frame number as
        # a long long, so unpack that.
        frame_raw = struct.unpack('Q', raw.recv())[0]
        frame_number = struct.unpack('Q', filt.recv())[0]

        # Grab the frame data from both raw and filtered channels.
        if filt.getsockopt(zmq.RCVMORE):
            filt_data = filt.recv()
        if raw.getsockopt(zmq.RCVMORE):
            raw_data = raw.recv()

        # Only filtered has spike events; ignore these.
        if filt.getsockopt(zmq.RCVMORE):
            events_data = filt.recv()

        # Looking for extra data.
        if raw.getsockopt(zmq.RCVMORE):
            #print(frame_number)
            while raw.getsockopt(zmq.RCVMORE):
                weird_data = raw.recv()
                #print(f'There was more data, size {len(weird_data)}.')

    except Exception as e:
        print(e)  # ! What does this do?
        continue

    # `frame_data` is a 1027-element array containing the recorded voltage
    # at each electrode, so unpack that into a usable format.
    filt_arr = np.array(array.array('f', filt_data))
    raw_arr = np.array(array.array('f', raw_data))

    # Don't bother reading the events in this one, just check the size adn
    # frame number of the two floating-point data frames.
    time_new =  round( time.time() )
    if time_old < time_new:
        print(f'Frame #{frame_number} (raw #{frame_raw}, Δ={frame_raw-frame_number}):')
        print(f'  raw has {len(raw_arr)} points, from {raw_arr.min()} to {raw_arr.max()}')
        print(f' filt has {len(filt_arr)} points, from {filt_arr.min()} to {filt_arr.max()}')
        time_old = time_new

# If you split this code into multiple threads, e.g. to consolidate
# this script with the one that handles the recordings, it will be
# necessary to delete each subscriber to the ZMQ context separately
# using the usual Python operator `del`, and then terminate the
# context with `ctx.term()`. In a single-threaded script like this,
# though, it's fine to just let the Python interpreter run all these
# destructors when the program terminates.

Run when finished so that we don't get an error message

In [ ]:
del filt 
del raw
ctx.term()

In [ ]:
len(raw_arr)

In [ ]:
plt.hist( raw_arr.tolist()[:30],3 )
plt.show()

In [ ]:
print(raw_arr.tolist())

In [ ]:
min(raw_arr)

In [ ]:
max(raw_arr)

In [ ]:
import numpy as np
from scipy import  stats
import pandas as pd

In [ ]:
print(stats.describe(raw_arr.tolist()))

In [ ]:
np.std(raw_arr)

In [ ]:
df = pd.DataFrame(raw_arr)
df.time_diff.quantile([0.25,0.5,0.75])

In [ ]:
np.quantile( raw_arr, .25 )

In [ ]:
np.quantile( raw_arr, .75 )